# Filters

[![open_in_colab][colab_badge]][colab_notebook_link]
[![open_in_binder][binder_badge]][binder_notebook_link]

[colab_badge]: https://colab.research.google.com/assets/colab-badge.svg
[colab_notebook_link]: https://colab.research.google.com/github/UnfoldedInc/examples/blob/master/notebooks/05%20-%20Filters.ipynb
[binder_badge]: https://mybinder.org/badge_logo.svg
[binder_notebook_link]: https://mybinder.org/v2/gh/UnfoldedInc/examples/master?urlpath=lab/tree/notebooks/05%20-%20Filters.ipynb

This example shows how to control map filters and listen to map filter change events with Unfolded Map SDK.

## Dependencies

This notebook requires the following Python dependencies:

- `unfolded.map-sdk`: The Unfolded Map SDK
- `pandas`: DataFrame library

If running this notebook in Binder, these dependencies should already be installed. If running in Colab, the next cell will install these dependencies.

In [1]:
# If in Colab, install this notebook's required dependencies
import sys
if "google.colab" in sys.modules:
    !pip install 'unfolded.map_sdk>=1.0' pandas

## Imports

In [2]:
from unfolded.map_sdk import create_map
import pandas as pd
from sidecar import Sidecar
import ipywidgets as widgets
from uuid import uuid4

## Using Map Filters

Let's again create a local map and add data to it:

In [3]:
unfolded_map = create_map()
sc = Sidecar(title='Unfolded Map', anchor='split-right')
with sc:
    display(unfolded_map)

In [4]:
url = 'https://raw.githubusercontent.com/UnfoldedInc/examples/master/notebooks/data/earthquakes.csv'
df = pd.read_csv(url)

In [5]:
dataset_id = str(uuid4())
unfolded_map.add_dataset({
    'id': dataset_id,
    'label': 'Earthquakes',
    'data': df
})

LocalDataset(id='3758e7d7-4f73-4d29-9d66-933c6ebedcb9', type=<DatasetType.LOCAL: 'local'>, label='Earthquakes', color=(143, 47, 191), fields=[BasicField(name='column_0', label='column_0', type=<BasicFieldType.INTEGER: 'integer'>), TimestampField(name='DateTime', label='DateTime', type=<TimestampFieldType.TIMESTAMP: 'timestamp'>, time_format='YYYY/M/D HH:mm:ss.SSSS'), BasicField(name='Latitude', label='Latitude', type=<BasicFieldType.REAL: 'real'>), BasicField(name='Longitude', label='Longitude', type=<BasicFieldType.REAL: 'real'>), BasicField(name='Depth', label='Depth', type=<BasicFieldType.REAL: 'real'>), BasicField(name='Magnitude', label='Magnitude', type=<BasicFieldType.REAL: 'real'>), BasicField(name='MagType', label='MagType', type=<BasicFieldType.STRING: 'string'>), BasicField(name='NbStations', label='NbStations', type=<BasicFieldType.INTEGER: 'integer'>), BasicField(name='Gap', label='Gap', type=<BasicFieldType.INTEGER: 'integer'>), BasicField(name='Distance', label='Distance

## Adding a filter

Say, we want to filter the data points by the `Magnitude` column. Let's first find out what's the extent of the values in this column:

In [6]:
magnitude_extent = [df['Magnitude'].min(), df['Magnitude'].max()]
magnitude_extent

[2.5, 7.39]

Now we can set the filter to only show the points in the top half of the range:

In [7]:
unfolded_map.add_filter({
    'id': 'magnitude_filter',
    'sources': [
        {
            'data_id': dataset_id,
            'field_name': 'Magnitude'
        }
    ], 
    'value': (
        (magnitude_extent[1] + magnitude_extent[0])/2,
        magnitude_extent[1]
    )
})

RangeFilter(id='magnitude_filter', type=<FilterType.RANGE: 'range'>, sources=[FilterSource(data_id='3758e7d7-4f73-4d29-9d66-933c6ebedcb9', field_name='Magnitude')], value=(4.945, 7.39))

Or the bottom half:

In [8]:
unfolded_map.update_filter(
    'magnitude_filter',
    value=(
        magnitude_extent[0],
        (magnitude_extent[1] + magnitude_extent[0])/2
    )
)

RangeFilter(id='magnitude_filter', type=<FilterType.RANGE: 'range'>, sources=[FilterSource(data_id='3758e7d7-4f73-4d29-9d66-933c6ebedcb9', field_name='Magnitude')], value=(2.5, 4.945))

## Controlling the filter from the notebook

We can use the range slider from `ipywidgets` to control the filter in the map. First, we create the slider:

In [9]:
slider = widgets.FloatRangeSlider(
    value=magnitude_extent,
    min=magnitude_extent[0],
    max=magnitude_extent[1],
    step=0.5,
    description='Magnitude:',
    continuous_update=False
)

Here we create an event handler to listen to the slider change events:

In [10]:
def update_value_filter(change):
    if 'new' in change and 'value' in change['new']:
        unfolded_map.update_filter(
            'magnitude_filter',
            value = tuple(change['new']['value'])
        )

In [11]:
slider.observe(update_value_filter)
slider

FloatRangeSlider(value=(2.5, 7.39), continuous_update=False, description='Magnitude:', max=7.39, min=2.5, step…

Now try moving the slider. You should see the changes applied to the map.

## Syncing the slider in the notebook with the map filter

We can register [event handlers](https://docs.unfolded.ai/map-sdk/api#events) to be notified of filter changes in the map. Here's how we can synchronize the above range slider with the Magnitude filter in the map:

In [ ]:
def on_filter_sync(info):
    if 'magnitude_filter' in info['id']:
        slider.value = info['value']

unfolded_map.set_event_handlers({
    'on_filter_update': on_filter_sync
})

Now try changing the "Magnitude" filter in the "Filter" pane of the left sidebar in the map. You should see the slider above in the notebook update. 

The following will unregister the observer:

In [ ]:
slider.unobserve(None)

##  Debugging an event handler

Here's how you can output the event info when filter events are triggered:

In [ ]:
output = widgets.Output()
@output.capture(clear_output=True)
def on_filter_output(info):
    print(info)
output

In [ ]:
unfolded_map.set_event_handlers({
    'on_filter_update': on_filter_output
})

Now when you change the filter in the Unfolded Studio map, you'll see above a printed object like:
```py
{'id': 'magnitude_filter', 'sources': [{'dataId': '95085e14-3531-4b77-a186-f53ec3311b6f', 'fieldName': 'Magnitude'}], 'type': 'range', 'value': [3.08, 4.29]}
```